In [1]:
import os
os.sys.path.append('../project_code')

from data.bhsa import book_sets, get_data
from markov.Markov_Chain import *
from PCA.PCA import * 

test = get_data(books=["lbh","sbh"])
features = {'clause_types', 'phrase_functions', 'phrase_types', 'word_pos'}
domain = {'N', 'Q', 'D'}
test.keys()

()


dict_keys([])

# Transitions

In [5]:
data = collections.defaultdict(dict)
for f in features:
    for d in domain:
        transMatrix = transitionMatrix(test[f][d], False) 
        #Here, transformations can be performed
        transDict = df_dict_Transformation(transMatrix)
        data[f][d] = pd.DataFrame(transDict).fillna(0)

In [6]:
data["phrase_functions"]["N"].head(10)

""


In [7]:
stack_Data = stackTransitions(data, features, "N")

# PCA

In [8]:
pca = PCA(n_components=2)

results_PCA = collections.defaultdict(lambda: collections.defaultdict(dict))
for f in features:
    for d in domain:
        if not data[f][d].empty:
            fit = pca.fit_transform(data[f][d].T.values) 
            results_PCA[f][d]["Fit"] = fit
            results_PCA[f][d]["var_Ratio"] = pca.explained_variance_ratio_

In [9]:
for f in features:
    for d in domain:
        if len(results_PCA[f][d].keys())>0: 
            fit = results_PCA[f][d]["Fit"]
            print(fit.shape)
            plt.figure(1, figsize=(15, 10))     
            plt.scatter(fit[:, 0], fit[:, 1])
            books = data[f][d].keys()
            for i, book in enumerate(books):
                plt.annotate(book, xy=(fit[i, 0], fit[i, 1]), fontsize=14)
            plt.show()

NameError: name 'sbh_books' is not defined